# Practical Exam Solution
---
Kisee S. Musau

## Data Validation
This data set has 6738 rows, 9 columns. I have validated all variables and I have not made any changes after validation. All the columns are just as described in the data dictionary:
- model: 18 models without missing values, same as the description. No cleaning is needed.
- year: 23 unique values without missing values, from 1998 to 2020, same as the description. No cleaning is needed.
- price: numeric values without missing values, same as the description. No cleaning is needed.
- transmission: 4 categories without missing values, same as the description. No cleaning is needed.
- mileage: numeric values, same as the description. No cleaning is needed.
- fuelType: 4 categories without missing values, same as the description. No cleaning is needed.
- mpg: numeric values without missing values, same as the description. No cleaning is needed.
- engineSize: 16 possible values without missing values, same as the description. No cleaning is needed.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score,mean_squared_error
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('data/toyota.csv')
df.info()

In [ ]:
df.head()

In [ ]:
#validate possible 18 values
df['model'].nunique()

In [ ]:
#validate year of manufacture from 1998 to 2020
df['year'].unique()

In [ ]:
#validate four types of transmission
df['transmission'].unique()

In [ ]:
#validate four fuel Types
df['fuelType'].unique()

In [ ]:
#validate 16 possible values in engineSize
df['engineSize'].nunique()

In [ ]:
#validate any negative values in numeric variables
df.describe()

## Exploratory Analysis
I have investigated the target variable and features of the car, and the relationship between target variable and features. After the analysis,I decided to apply the following changes to enable modeling:
- Price: use log transformation
- Create a new ordinal variable from tax variable

### Target Variable - Price

Since we need to predict the price, the price variable would be our target variable. From the histogram on the left below, we can see there is a longer right tail. Therefore, we apply log transforamtion of the price variable, the distribution on the right below is close to normal distribution.

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(15,5))
sns.histplot(df['price'],color='gray', ax=axes[0]).set(title='The Distribution of Target Variable - Price')
sns.histplot(df['price'],log_scale=True,color='gray', ax=axes[1]).set(title='The Distribution of Target Variable - Price (Log Scale)');

### Numeric Variables - Mileage, Tax, mpg

From the heatmap below, we can conclude that there is a moderate linear negative relationship in two pairs of variables - price log transformation and mileage, tax and mpg.

In [ ]:
numeric = df[['price','mileage','tax','mpg']]
sns.heatmap(numeric.corr(),annot=True).set(title='The Correlation Heatmap between Numeric Variables');

#### Relationship between mpg, tax, mileage and price
To spot the non-linear relationship, I decided to make scatterplot to further investigate the relationship between mpg, tax, mileage and our target varible - price. From the scatterplots below, there is linear relationship between mileage and price. No relationship between price and mpg. I found out there is clusters in the scatterplot between price and tax, so I decided to create a new ordinal variable from the tax variable.

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(15,5))
sns.scatterplot(y=df['price'],x=df['mpg'],color='gray',ax=axes[0]).set(title='Price vs mpg')
sns.scatterplot(y=df['price'],x=df['tax'],color='gray',ax=axes[1]).set(title='Price vs tax')
sns.scatterplot(y=df['price'],x=df['mileage'],color='gray',ax=axes[2]).set(title='Price vs Mileage');

In [ ]:
# Convert tax variable into an ordinal variable  
df.loc[(df['tax'] <= 100,'tax')] = 1
df.loc[((df['tax'] <= 200) & (df['tax'] > 100) ,'tax')] = 2
df.loc[((df['tax'] <= 300) & (df['tax'] > 200) ,'tax')] = 3
df.loc[(df['tax'] > 300 ,'tax')] = 4

### Categorical Variables - Year, Engine Size, Model, Transmission, fuelType
#### Characteristics about Year, Engine size, Model, Transmission, and fuelType
Since year and engine size is most related to price, I checked their characteristics. From the bar chart below, the most common manufacture year is 2016. Also, the most common engine size in is 1.

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(15,5))
sns.countplot(x=df['year'], color='gray',ax=axes[0]).set(title='Count of Cars Sold in Manufacture Year')
sns.countplot(x=df['engineSize'],color='gray',ax=axes[1]).set(title='Count of Cars Sold in EngineSize')
axes[0].tick_params(axis='x', labelrotation=45)
axes[1].tick_params(axis='x', labelrotation=45);

From the bar charts below, we can see the most frequent categories in model, transmission and fuelType variables - Yaris, Manual, Petrol in the dataset. 

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(15,5))
sns.countplot(x=df['model'],color='gray',ax=axes[0]).set(title='Count of Cars Sold in Model')
sns.countplot(x=df['transmission'],color='gray',ax=axes[1]).set(title='Count of Cars Sold in Transmission')
sns.countplot(x=df['fuelType'],color='gray',ax=axes[2]).set(title='Count of Cars Sold in Fuel Type')
for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90);

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(20,5))
sns.scatterplot(data=df, x='year',y='price',color='gray', ax=axes[0]).set(title='The Relationship Between Year and Price')
sns.boxplot(data=df, x='engineSize',y='price',color='gray', ax=axes[1]).set(title='The Distribution of Price by Engine Size')
for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90);

In [ ]:
sns.scatterplot(data=df, x='year',y='price',color='gray', hue='engineSize').set(title='The Relationship Between Manufacture Year, Price and Engine Size');

#### Relationship between Model, Transmission, fuelType and price

I also investigated their relationship between price. From the boxplots below, we can see there is a difference in distribution of prices among each categories in each variable.

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(20,5))
sns.boxplot(data=df, x='model',y='price',color='gray',ax=axes[0]).set(title='The Distribution of Price by Model')
sns.boxplot(data=df, x='transmission',y='price',color='gray',ax=axes[1]).set(title='The Distribution of Price by Transmission')
sns.boxplot(data=df, x='fuelType',y='price',color='gray',ax=axes[2]).set(title='The Distribution of Price by Fuel Type')
for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90);

## Model Fitting & Evaluation
Predicting the price is a regression problem in machine learning. I am choosing the Linear Regression model because we can see strong to moderate relationship between some features and target variable. The comparison model I am choosing is the Decision Tree regression model because it is easy to interpret with independence from outliers.

For the evaluation, I am choosing **R squared** and **RMSE (Root Mean Squared Error)** to evaluate the model. R squared measures how well the model fits dependent variables (i.e. features). RMSE measures how much your predicted results deviate from the actual number. 

### Prepare Data for Modelling
To enable modelling, we chose year,model,transmission,mileage,fuelType,tax,engineSize as features, price as target variables. I also have made the following changes:
- Normalize the numeric features
- Convert the categorical variables into numeric features
- Split the data into a training set and a test set

In [ ]:
df['price'] = np.log(df['price'])

In [ ]:
labelencoder = LabelEncoder()
df['model'] = labelencoder.fit_transform(df['model'])
df['transmission'] = labelencoder.fit_transform(df['transmission'])
df['fuelType'] = labelencoder.fit_transform(df['fuelType'])

In [ ]:
feature_cols = ['year','transmission','fuelType','engineSize','tax','model','mileage']
X = df[feature_cols] # Features
y = df['price'] # Target variable

In [ ]:
# define the scaler 
scaler = PowerTransformer()
# fit and transform the train set
X[['mileage']] = scaler.fit_transform(X[['mileage']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Linear Regression Model

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

print('Linear Regression r2_score: ',r2_score(y_test,y_pred))
print('Linear Regression Root Mean Squared Error: ',np.sqrt(mean_squared_error(np.exp(y_test),np.exp(y_pred))))

#### Finding the feature importance

In [ ]:
resultdict = {}
for i in range(len(feature_cols)):
    resultdict[feature_cols[i]] = lr.coef_[i]
    
plt.bar(resultdict.keys(),resultdict.values(), alpha=0.5,color='gray')
plt.xticks(rotation='vertical')
plt.title('Feature Importance in Linear Regression Model');

### Decision Tree Regression Model

In [ ]:
tree = DecisionTreeRegressor(max_depth=12,min_samples_split=2,random_state=42)
tree.fit(X_train,y_train)
y_pred2 = tree.predict(X_test)

In [ ]:
d_r2 = tree.score(X_test, y_test)
print("Decision Tree Regressor R-squared: {}".format(d_r2))

d_mse = mean_squared_error(np.exp(y_pred2), np.exp(y_test))
d_rmse = np.sqrt(d_mse)
print("Decision Tree Regressor RMSE: {}".format(d_rmse))

#### Finding the best parameter for Decision Tree Regression Model

In [ ]:
train_score = []
test_score = []
max_score = 0
max_pair = (0,0)

for i in range(1,50):
    tree = DecisionTreeRegressor(max_depth=i,random_state=42)
    tree.fit(X_train,y_train)
    y_pred = tree.predict(X_test)
    train_score.append(tree.score(X_train,y_train))
    test_score.append(r2_score(y_test,y_pred))
    test_pair = (i,r2_score(y_test,y_pred))
    if test_pair[1] > max_pair[1]:
        max_pair = test_pair

fig, ax = plt.subplots()
ax.plot(np.arange(1,50), train_score, label = "Training R^2",color='lightcoral')
ax.plot(np.arange(1,50), test_score, label = "Testing R^2",color='lime')
print(f'Best max_depth is: {max_pair[0]} \nTesting R^2 is: {max_pair[1]}')

#### Finding the feature importance

In [ ]:
importance = tree.feature_importances_
new_column_names=['Year','Transmission', 'Fuel Type','Engine Size','Tax','Model','Mileage']
f_importance = {}
for i in range(len(feature_cols)):
     f_importance[new_column_names[i]] = importance[i]
        
plt.bar(f_importance.keys(),f_importance.values(),alpha=0.5,color='gray')
plt.xticks(rotation=45)
plt.title('Feature Importance in Decision Tree Regression Model');

## Results
The R squared of the Linear Regression, and the Decision Tree Regression model is 0.86 and 0.94, meaning the **Decision Tree Regression model fits the features better.** The RMSE of the Linear Regression, and the Decision Tree Regression model is 2491 and 1611, meaning the **Decision Tree Regression model has less error in predicting values.**

### Evaluate by Business Criteria
The company wants to avoid prediction out of a range - within 10% of the actual price. Therefore, we would consider using percentage of predictions within 10% of the actual price as a KPI to compare the two models again. The higher the percentage, the better the model performs. 69.4% of the decision tree regressor prediction is within 10% of the actual price, while the linear regression model only have 61.8%.

In [ ]:
X_test['Predicted_price'] = np.round(np.exp(y_pred),0)
X_test['Price'] = np.round(np.exp(y_test),0)
lr_e = X_test
lr_e['Diff'] = (lr_e['Predicted_price'] - lr_e['Price'])/lr_e['Price']
within_10pct = len([d for d in lr_e['Diff'] if abs(d) <= 0.1])
pct_lr = within_10pct/len(lr_e['Diff'])       
print(f'Percentage of predictions within +/- 10% range: {pct_lr}')

In [ ]:
lr_e['Diff'] = (lr_e['Predicted_price'] - lr_e['Price'])/lr_e['Price']
plt.hist(lr_e['Diff'], bins=20, color='gray',alpha=0.5)
plt.axvline(x=0.1, color='b', linestyle='--', label='+/- 10% range')
plt.axvline(x=-0.1, color='b', linestyle='--')
plt.title('Percentage Difference between Actual and Predicted Prices')
plt.suptitle('Linear Regression Model')
plt.xlabel('Percentage Difference')
plt.ylabel('Count');

In [ ]:
X_test['Predicted_price'] = np.round(np.exp(y_pred2),0)
X_test['Price'] = np.round(np.exp(y_test),0)
tree_e = X_test
tree_e['Diff'] = (tree_e['Predicted_price'] - tree_e['Price'])/tree_e['Price']
within_10pct = len([d for d in tree_e['Diff'] if abs(d) <= 0.1])
pct_tree = within_10pct/len(tree_e['Diff'])       
print(f'Percentage of predictions within +/- 10% range: {pct_tree}')

In [ ]:
plt.hist(tree_e['Diff'], bins=20,color='gray', alpha=0.5)
plt.axvline(x=0.1, color='b', linestyle='--', label='+/- 10% range')
plt.axvline(x=-0.1, color='b', linestyle='--')
plt.title('Percentage Difference between Actual and Predicted Prices')
plt.suptitle('Decision Tree Model')
plt.xlabel('Percentage Difference')
plt.ylabel('Count');

In [ ]:
da = [pct_lr,pct_tree]
labels =['Linear Regression Model','Decision Tree Model']
plt.bar(labels, da,color='gray', alpha=0.5)


# Add labels to the chart
plt.title('Linear Regress Model v.s Decision Tree Model')
plt.xlabel('Model')
plt.ylabel('Proportion of predictions within 10%')

# Show the chart
plt.show()

## Recommendation 

To smooth out the difficulities we might face when the most experienced sales team member retires, we can plan to deploy this **Decision Tree Regression Model** into production. By implementing this model, about 70% of the prediction will within 10% of the price. This will help automate the whole process and sell cars quicker. I will recommend the following steps to ensure the model can be deployed and improved regularly.

1. Test the model in the recent month and compare its prediction to the estimations of the experienced member on new cars.

2. Identify and fix errors to improve accuracy after testing.

3. Fully deploy the model using efficient deployment strategy, such as a web or mobile application or API.

4. Continuously improve the model by collecting more data, feature engineer and fine tuning parameter. This would be the key points to address the limitation from the year and car model variable.

